### Importing necessary libraries

In [1]:
import cv2
import numpy as np
import os
import random
import glob
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

## Image Analysis

### Pre-Processing of Images

In [2]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    
    # Resize the image to 128x128
    down_width = 128
    down_height = 128
    down_points = (down_width, down_height)
    resize_img = cv2.resize(img, down_points)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(resize_img, cv2.COLOR_BGR2GRAY)
    
    # Apply a sharpening filter
    kernel1 = np.array([[0, -1,  0],
                        [-1,  5, -1],
                        [0, -1,  0]])
    sharp_img = cv2.filter2D(src=gray, ddepth=-1, kernel=kernel1)
    
    # Apply thresholding to convert the image into a binary image
    _, binary = cv2.threshold(sharp_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Apply morphological operations to remove noise and fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Find the contours in the binary image
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw the contours on the original image
    cv2.drawContours(resize_img, contours, -1, (0, 255, 0), 2)
    
    return resize_img

## The Machine Learning Model

##### Loading the dataset

In [3]:
path = "F:\Mini Project\DR Dataset"
target_names = ['Severe DR', 'Healthy']
data = []
target = []

# load the image data and labels
for target_index, target_name in enumerate(target_names):
    for img_name in os.listdir(os.path.join(path, target_name)):
        img_path = os.path.join(path, target_name, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = preprocess_image(img_path)
        data.append(img)
        target.append(target_index)
print("Data size:", len(data))
print("Target size:", len(target))

Data size: 1190
Target size: 1190


##### Converting data and target lists into numpy arrays

In [4]:
data = np.array(data)
target = np.array(target)

##### Spliting the data into training and testing sets

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

##### Initializing the model and fitting the training data

In [6]:
model = SVC(kernel='linear')
model.fit(X_train.reshape(len(X_train), -1), y_train)

SVC(kernel='linear')

##### Predicting for new image

In [10]:
new_img_path = "Testing\Healthy_5.png"
new_image = preprocess_image(new_img_path)
new_img_1d = new_image.reshape(1, -1)
prediction = model.predict(new_img_1d)[0]
if prediction == 0:
    print(f'The input image {new_img_path} is Defective')
else:
    print(f'The input image {new_img_path} is Not Defective')

The input image is Not Defective


##### Evaluating model on the testing data

In [8]:
accuracy = model.score(X_test.reshape(len(X_test), -1), y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9243697478991597


##### Model testing images randomly from base path

In [9]:
# Select a random image from the base folder/path
random_image_path = random.choice(glob.glob(f"{path}/**/*.png", recursive=True))

# Preprocess the image
new_image = preprocess_image(random_image_path)
new_img_1d = new_image.reshape(1, -1)

# Perform prediction
prediction = model.predict(new_img_1d)[0]

# Map the prediction to the corresponding class label
prediction_label = target_names[prediction]

# Print the result
print("Randomly selected image path:", random_image_path)
print("Prediction:", prediction_label)

if prediction == 0:
    print("Diabetic Retinopathy found.")
else:
    print("Diabetic Retinopathy not found.")


Randomly selected image path: F:\Mini Project\DR Dataset\Healthy\Healthy_509.png
Prediction: Healthy
Diabetic Retinopathy not found.
